# 1) Galaxia 3-field data

* [Check fileinfo for the Galaxia data](#check)
* [Sort and save the field pointings file](#field)
* [Run field assignment on Galaxia data](#fa)
* [Generate the isochrone calculator](#iso)
* [Correct photometric files](#photo)
* [Correct spectroscopic files](#spectro)
* [Apply selection function criteria](#SF)

Saved version of the repository online is [here](https://drive.google.com/file/d/1XLFX1uRS3INFsdL3s889SuBtw1k3W-11/view?usp=sharing)

Run 
```tar -xvf Galaxia3_new.tar.gz```
to untar and unzip file.

In [1]:
# autoreload modules when files have been altered
%load_ext autoreload
%autoreload 2

import sys, os
import ebf
import numpy as np
import pandas as pd
import dill

%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib
from matplotlib.colors import LogNorm
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 30}
matplotlib.rc('font', **font)

import seaborn as sns
rc={'font.size': 16, 'axes.labelsize': 16, 'legend.fontsize': 12,
    'axes.titlesize': 18, 'xtick.labelsize': 12, 'ytick.labelsize': 12}
sns.set(font="serif",style="white",rc=rc)

def ErrorWarning():
    raise ValueError("Wait, don't run this, you don't want to!")

<a name="check"></a>
# File info

In [2]:
from seestar import surveyInfoPickler
fileinfo = surveyInfoPickler.surveyInformation('PATH/Galaxia3_new/Galaxia3_new_fileinfo.pickle')

In [3]:
fileinfo.testFiles()

1) Checking file paths exist:
OK

2) Checking spectroscopic catalogue file structure:
OK

3) Checking field information file structure:
(make sure halfangle is in units of radians.)
OK

4) Checking photometric catalogue file structure:
Checking solutionF1.csv:
OK

5) Checking selection function pickle paths exist:
OK

6) Checking isochrone pickle files exist:
The premade interpolants (isochrone_interpolantinstances.pickle) will be automatically be used to calculate the selection function.


In [5]:
fileinfo.save()

<a name="field"></a>
# Field info file

In [16]:
df = pd.read_csv('PATH/Galaxia3/Galaxia3_fieldinfo.csv')
df.glon *= np.pi/180
df.glat *= np.pi/180
df.drop('Unnamed: 0', axis=1, inplace=True)
df.rename(index=str, columns={'hmin':'Magmin', 'hmax':'Magmax', 'cmin':'Colmin', 'cmax':'Colmax'}, inplace=True)
df.to_csv(fileinfo.field_path, index=False)

<a name="fa"></a>
# Field Assignment

In [2]:
from seestar import FieldAssignment

In [24]:
fileinfo_path = 'PATH/Galaxia3_new/Galaxia3_new_fileinfo.pickle'
# Galaxia 'photometric' files
files = ["PATH/Galaxia3/Galaxy3_fieldassign_"+str(n)+".csv" for n in range(1, 15)]
FA = FieldAssignment.FieldAssignment(fileinfo_path, files)
# ncores=1: SF will run the Serial version (not use multiprocessing)
# ncores>1: Parallel version on the number of cores given
# When running in parallel, the field counting can mess up a bit because the pools get disordered

# You can also specify memory= as an additional kwargs
# If memory given: its currently set up to use 5% of given memory
# If no memory kwarg given: It will use 5% of the memory available in the computer (not sure how this works on a cluser)

Checking photometric catalogue file structure:
Checking /media/andy/UUI/Research/SF/SFdata/Galaxia/Galaxia3/Galaxy3_fieldassign_2.csv:
Datatype of column glon given as <type 'numpy.object_'> but is actually <type 'numpy.float64'>.
Changed dtype to <type 'numpy.float64'>, run self.save() to keep these changes.
Datatype of column glat given as <type 'numpy.object_'> but is actually <type 'numpy.float64'>.
Changed dtype to <type 'numpy.float64'>, run self.save() to keep these changes.
Datatype of column Japp given as <type 'numpy.object_'> but is actually <type 'numpy.float64'>.
Changed dtype to <type 'numpy.float64'>, run self.save() to keep these changes.
Datatype of column Kapp given as <type 'numpy.object_'> but is actually <type 'numpy.float64'>.
Changed dtype to <type 'numpy.float64'>, run self.save() to keep these changes.
Datatype of column Happ given as <type 'numpy.object_'> but is actually <type 'numpy.float64'>.
Changed dtype to <type 'numpy.float64'>, run self.save() to keep 

/home/andy/Documents/Research/SF/GitRepo/seestar/FieldAssignment.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df.drop('points', inplace=True, axis=1)


allsky file: Galaxy3_fieldassign_14.csv  Completion: 66853156/66853170(100.0%)  Time=12.8m  Projected: 0h12m           

<a name="iso"></a>
# Isochrone calculator (Just for Galaxia)

In [2]:
from seestar import IsochroneScaling

In [4]:
fileinfo.iso_data_path, fileinfo.iso_interp_path, fileinfo.iso_mag_path

('/media/andy/37E3-0F91/Research/SF/SFdata/Galaxia3_new/isochrones/stellarprop_parsecdefault_currentmass.dill',
 '/media/andy/37E3-0F91/Research/SF/SFdata/Galaxia3_new/isochrones/isochrone_interpolantinstances.pickle',
 '/media/andy/37E3-0F91/Research/SF/SFdata/Galaxia3_new/isochrones/isochrone_magnitudes.pickle')

In [5]:
ErrorWarning()
IsoCalculator = IsochroneScaling.IntrinsicToObservable()
iso_data = '.../isoPARSEC/stellarprop_parsecdefault_currentmass.dill'
IsoCalculator.CreateFromIsochrones(iso_data)
IsoCalculator.pickleColMag(fileinfo.iso_interp_path)

/media/andy/37E3-0F91/Research/SF/SFdata/isoPARSEC/stellarprop_parsecdefault_currentmass.dill
Undilling isochrone interpolants...
...done.



/home/andy/anaconda2/lib/python2.7/site-packages/seestar-1.0-py2.7.egg/seestar/IsochroneScaling.py:639: RuntimeWarning: divide by zero encountered in double_scalars


Scaled masses spacing: 352 / 353
Scaled masses points: 999 / 1000
nage, nmh, nmass: 353, 54, 500
((0.0, 13.220006551228417), (-2.2435283018867924, 0.5905283018867925), (0.0, 1.0))


In [10]:
IsoCalculator = IsochroneScaling.IntrinsicToObservable()
IsoCalculator.LoadColMag(fileinfo.iso_interp_path)

<a name="photo"></a>
## Correct Photometric files

In [25]:
pointings = pd.read_csv(fileinfo.field_path)
for field in pointings.fieldID:
    
    filepath = os.path.join(fileinfo.photo_path, str(field))+'.csv'
    
    df = pd.read_csv(filepath)
    
    # Apparent magnitudes
    df['j'], df['k'], df['h'] = IsoCalculator.AppMags(df.ageGyr, df.feh, df.smass, df.rad)
    
    stats =(len(df), len(df[pd.notnull(df.h)]), len(df[pd.notnull(df.j)]))
    print(stats)
    df = df[pd.notnull(df.h)]
    
    df = df[['fieldID', 'ageGyr', 'feh', 'smass', 'rad', 'glat', 'glon', 'j', 'k', 'h']]
    
    df.rename(index=str, columns={'feh':'mh', 'smass':'mass', 'rad':'s',
                                      'j':'Japp', 'k':'Kapp', 'h':'Happ'}, inplace=True)
    
    df.to_csv(filepath, index=False)

(2625266, 1969303, 1969303)
(8655, 7340, 7340)
(8465, 7226, 7226)


<a name="spectro"></a>
## Correct Spectroscopic files

In [16]:
ErrorWarning()
# No longer using this because we need to do the selection criteria in the next cell.

survey = pd.read_csv('PATH/Galaxia3/Galaxia3_survey.csv')

survey['j'], survey['k'], survey['h'] = IsoCalculator.AppMags(survey.ageGyr, survey.feh, survey.smass, survey.rad)

stats = (len(survey), len(survey[pd.notnull(survey.h)]), len(survey[pd.notnull(survey.j)]))
print(stats)
survey = survey[pd.notnull(survey.h)]

survey = survey[['fieldid', 'ageGyr', 'feh', 'smass', 'rad', 'glat', 'glon', 'j', 'k', 'h']]

survey.rename(index=str, columns={'fieldid':'fieldID', 'feh':'mh', 'smass':'mass', 'rad':'s',
                                  'j':'Japp', 'k':'Kapp', 'h':'Happ'}, inplace=True)

survey.glat *= np.pi/180
survey.glon *= np.pi/180

survey.to_csv(fileinfo.spectro_path, index=False)

(8885, 6263, 6263)


<a name="SF"></a>
## Selection criteria for spectroscpoic files

In [24]:
pd.read_csv(fileinfo.field_path)

,glon,glat,SA,halfangle,fieldID,Magmin,Magmax,Colmin,Colmax
0,0.523599,0.000000,12.565095,0.034907,1.0,8.0,13.5,0.5,NoLimit
1,0.523599,1.047198,12.565095,0.034907,2.0,8.0,13.5,0.5,NoLimit
2,0.523599,1.064651,12.565095,0.034907,3.0,8.0,13.5,0.5,NoLimit


In [57]:
survey_df = pd.DataFrame()

for index, row in pointings.iterrows():
    
    filepath = os.path.join(fileinfo.photo_path, str(row.fieldID))+'.csv'
    fieldstars = pd.read_csv(filepath)
    
    colour = fieldstars.Japp - fieldstars.Kapp
    Happ = fieldstars.Happ
    fieldstars = fieldstars[(Happ>row.Magmin) & (Happ<row.Magmax) & (colour>row.Colmin)]
    
    if row.fieldID == 1.0: fraction = 0.02
    else: fraction = 0.1
        
    rand_select = np.random.rand(len(fieldstars))
    fieldstars = fieldstars[rand_select<fraction]
    
    fieldstars['fieldID'] = row.fieldID

    survey_df = pd.concat((survey_df, fieldstars))
    
survey_df.to_csv(fileinfo.spectro_path, index=False)